In [1]:
import cv2
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import xml.etree.ElementTree as ET
import os
from sklearn.model_selection import train_test_split
import torch


In [2]:
os.chdir(r"C:\Users\Lucas\Desktop\vacantion classes\Yolo project\classes")
from CNN import CNN_model #import my class CNN

In [3]:
os.chdir(r"C:\Users\Lucas\Desktop\vacantion classes\Yolo project\data\Pascal_VOC\train")

In [4]:
path = r"C:\Users\Lucas\Desktop\vacantion classes\Yolo project\data\Pascal_VOC\train"

xml_list = []
for filename in os.listdir(path):
    if not filename.endswith('.xml'): continue
    fullname = os.path.join(path, filename)
    xml_list.append(fullname)

In [5]:
my_CNN = CNN_model()
my_CNN.get_annotation(xml_list)
my_CNN.encode_pictures()

In [6]:
os.chdir(r"C:\Users\Lucas\Desktop\vacantion classes\Yolo project")
my_CNN.df.to_json(r"data\annot_df.json",orient="records")
torch.save(my_CNN.encoded_picture_annot, r"data\encoded_picture.pt")

In [7]:
my_CNN.encoded_picture_annot

,file_name,encoded_grid
1,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
2,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
3,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
4,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
5,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
...,...,...
13295,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
13296,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
13297,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
13298,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."


In [8]:
df = pd.read_json(r"data\annot_df.json")
df

,file_name,ID_class,x_center,y_center,width,height
0,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,0,0.479000,0.464413,0.542000,0.373665
1,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,0,0.330000,0.375445,0.128000,0.124555
2,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,1,0.408000,0.727758,0.036000,0.174377
3,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,1,0.070000,0.759786,0.036000,0.174377
4,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,0,0.508000,0.505464,0.980000,0.426230
...,...,...,...,...,...,...
31351,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,1,0.740000,0.541333,0.500000,0.917333
31352,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,1,0.342988,0.635000,0.326220,0.390000
31353,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,1,0.742378,0.558000,0.149390,0.212000
31354,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,1,0.512012,0.485000,0.681682,0.618000


In [9]:
df_encoded = torch.load(r"data\encoded_picture.pt")
df_encoded

,file_name,encoded_grid
1,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
2,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
3,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
4,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
5,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
...,...,...
13295,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
13296,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
13297,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
13298,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."


import cv2
import numpy as np
import torch

S = 7
B = 2
C = len(my_CNN.classes)  # ton nombre de classes

X_train = []
Y_train = []

for idx, row in my_CNN.encoded_picture_annot.iterrows():
    if idx > 5000:
        break
    # Charger l'image
    img_path = row['file_name']
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (448,448))  # taille YOLO standard
    img = img / 255.0  # normalisation
    X_train.append(img)

    # Convertir le y_true de tensor à numpy
    y_tensor = row['encoded_grid']  # c'est un tensor ou liste de tensors
    if isinstance(y_tensor, torch.Tensor):
        y_numpy = y_tensor.numpy()
    else:
        # Si c'est une liste de listes de tensors
        y_numpy = np.array([[t.item() for t in line] for line in y_tensor])
    Y_train.append(y_numpy)

X_train = np.array(X_train, dtype=np.float16)
Y_train = np.array(Y_train, dtype=np.float16)

print(X_train.shape, Y_train.shape)

## Model

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df_encoded['file_name'],df_encoded['encoded_grid'],test_size=0.2)

X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

## Data_generator

We create a **batch generator** that will give one batch at a time to our model, avoiding to stock all our data at the same time, and thus **saving memory**.

In [11]:
def data_generator(X_paths, Y_tensor, batch_size=8):
    while True: #needed if epochs from model.fit is superior to 1
        for start in range(0, len(X_paths), batch_size):
            end = min(start + batch_size, len(X_paths))
            batch_X = []
            batch_Y = []

            for i in range(start, end):
                # load and preprocess image
                img = cv2.imread(X_paths[i])
                img = cv2.resize(img, (448, 448))
                img = img / 255.0
                batch_X.append(img.astype(np.float32))

                # convert label
                if isinstance(Y_tensor, torch.Tensor):
                    y_numpy = Y_tensor[i].cpu().numpy() #add it to cpu if on tensor on GPU and turn it into a numpy array
                else:
                    y_numpy = np.array(Y_tensor[i])
                batch_Y.append(y_numpy.astype(np.float32))

            yield np.array(batch_X), np.array(batch_Y)  #our results are converted to arrays because .fit from tensorflow only understand tf-tensor or numpy array
                                                        #What's more it only if it gets tf-tensor it will internally turn them into np array, 
                                                        #so we directly turn them into array to gain time

### Getting the loss function

In [12]:
def yolo_loss(y_true, y_pred):
    return my_CNN.yolo_loss_tf(y_true, y_pred, S=my_CNN.S, B=my_CNN.B, C=len(my_CNN.classes))

In [13]:
model = my_CNN.Yolo1(C=len(my_CNN.classes))  # your model
model.compile(optimizer='adam', 
              loss=yolo_loss,
              metrics=[])

In [14]:
batch_size = 16
train_gen = data_generator(X_train, y_train, batch_size=batch_size)
val_gen = data_generator(X_test, y_test, batch_size=batch_size)

steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_test) // batch_size

history = model.fit(
    train_gen,
    epochs=1,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_gen,
    validation_steps=validation_steps
)

model.save(r"model\YoloV1")


664/664 [==============================] - 3484s 5s/step - loss: 134.7581 - val_loss: 119.9515
INFO:tensorflow:Assets written to: model\YoloV1\assets


In [15]:
#history = model.fit(X_train, Y_train, epochs=1, batch_size=8, validation_split=0.1) #only trained on 1000 pictures

In [16]:
loaded_model = tf.keras.models.load_model(r"model\YoloV1", custom_objects={'yolo_loss': yolo_loss})


In [17]:
def get_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (448, 448))
    img = img / 255.0
    return img.astype(np.float32)

In [26]:
img = get_image(X_test[4])
img = np.expand_dims(img, axis=0)  # shape becomes (1, 448, 448, 3)
predictions = loaded_model.predict(img)

In [27]:
def display_predict(tensor,S,B,C):
    boxes_list = [] 
    for i in range(S):
        for j in range(S):
            boxes = tensor[i,j, :B*5]           # select : class, x_center, y_center, width, height
            class_probs = tensor[i,j, B*5:]     # select prob of each class 
            class_probs = 1 / (1 + np.exp(-class_probs)) # class probabilities in [0,1]
            for b in range(B):
                cls, x, y, w, h = boxes[b*5:(b+1)*5]
                print("w,h",w,h)
                img_h, img_w = 448, 448
                x_img ,y_img, w_img, h_img= (j + x) * img_w / S, (i + y) * img_h / S, (w) * img_w, abs(h) * img_h
                cls = 1 / (1 + np.exp(-cls))                # confidence in [0,1]
                class_id = np.argmax(class_probs)
                class_prob = class_probs[class_id]
                final_score = cls * class_prob
                
                boxes_list.append({
                    'cell': (i, j),
                    'bbox': (x_img, y_img, w_img, h_img),
                    'class_id': class_id,
                    'score': final_score
                })  
    
    boxes_list = [b for b in boxes_list if b['score'] > 0.3]

    img = cv2.imread(X_test[0])
    img = cv2.resize(img, (448, 448))

    for b in boxes_list:
        x, y, w, h = b['bbox']
        print(x,y,w,h)
        class_id = b['class_id']
        score = b['score']
        
        cv2.rectangle(img, (int(x - w/2), int(y - h/2)), (int(x + w/2), int(y + h/2)), (0,255,0), 2)
        cv2.putText(img, f'{my_CNN.classes[class_id]}:{score:.2f}', (int(x-w/2), int(y-h/2)-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)

    cv2.imshow('YOLO prediction', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return

In [28]:
display_predict(predictions[0],my_CNN.S,my_CNN.B,my_CNN.C)

w,h 3.6262136e-05 -0.0002483672
w,h 0.0003918603 5.9710743e-05
w,h 0.0011377905 0.00016393911
w,h 0.0007089384 -0.001051821
w,h -0.0003209729 0.00022504578
w,h 0.0012730812 0.00045947003
w,h -0.00058495876 0.00036279162
w,h 0.0006045221 -0.00068492955
w,h -0.001212699 0.00054904644
w,h 0.0001784771 -0.00025220352
w,h -0.00015693472 -0.00031903497
w,h 0.0004809468 0.0012152724
w,h 0.00093286496 -8.592846e-05
w,h 0.0014568289 -0.00017882066
w,h 0.00022096754 -0.0003831884
w,h 0.0007962146 0.0013805341
w,h 0.0010597125 0.00057831465
w,h -0.0005222794 0.0012317448
w,h 0.001864034 0.00021641375
w,h -0.00027979683 -0.001937701
w,h 0.00073215633 0.00042423065
w,h -0.00023457137 0.000566661
w,h -0.0005302388 -6.1868923e-06
w,h -0.00024909235 -0.0002223582
w,h 9.40141e-05 0.00049350807
w,h 1.9413812e-05 -0.00055933156
w,h -0.0002910086 -0.0011899871
w,h 0.00013827656 0.00090788055
w,h 0.00033372798 0.0015702539
w,h 0.000866944 3.0790765e-05
w,h -0.009872589 0.0010810157
w,h 0.00028710536 -0.000